In [26]:
import pandas as pd
import numpy as np

In [27]:
df=pd.read_csv("C:/Users/JUPALLY MEGHANA/Downloads/archive/European Restaurant Reviews.csv")

In [28]:
df.shape

(1502, 6)

In [29]:
df.head()

,Country,Restaurant Name,Sentiment,Review Title,Review Date,Review
0,France,The Frog at Bercy Village,Negative,Rude manager,May 2024 •,The manager became agressive when I said the c...
1,France,The Frog at Bercy Village,Negative,A big disappointment,Feb 2024 •,"I ordered a beef fillet ask to be done medium,..."
2,France,The Frog at Bercy Village,Negative,Pretty Place with Bland Food,Nov 2023 •,"This is an attractive venue with welcoming, al..."
3,France,The Frog at Bercy Village,Negative,Great service and wine but inedible food,Mar 2023 •,Sadly I used the high TripAdvisor rating too ...
4,France,The Frog at Bercy Village,Negative,Avoid- Worst meal in Rome - possibly ever,Nov 2022 •,From the start this meal was bad- especially g...


In [30]:
df['Review']

0       The manager became agressive when I said the c...
1       I ordered a beef fillet ask to be done medium,...
2       This is an attractive venue with welcoming, al...
3       Sadly I  used the high TripAdvisor rating too ...
4       From the start this meal was bad- especially g...
                              ...                        
1497    Despite the other reviews saying that this is ...
1498    beer is good.  food is awfull  The only decent...
1499    for terrible service of a truly comedic level,...
1500    We visited the Havana's Club Museum which is l...
1501    Food and service was awful. Very pretty stop. ...
Name: Review, Length: 1502, dtype: object

In [31]:
df['Review'][0]

'The manager became agressive when I said the carbonara was not good. She was rude. It was  a very bad experience. The worst in Rome in many years.'

In [32]:
import nltk
nltk.download()



showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [36]:
from nltk.book import *

In [37]:
import re
from nltk.corpus import stopwords

In [38]:
from nltk.stem.porter import PorterStemmer

In [39]:
stop_words=set(stopwords.words('english'))

In [42]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

def preprocessing_text(txt):
    # Lower case
    txt = txt.lower()
    # Remove HTML tags
    txt = re.sub(r"<.*?>", " ", txt)
    # Remove special characters and digits
    txt = re.sub(r"[^a-zA-Z]", " ", txt)
    # tokenization
    txt = nltk.word_tokenize(txt)
    # Remove stopwords
    txt = [word for word in txt if word not in stop_words]
    # Remove words less than three letters
    txt = [word for word in txt if len(word) >= 3]
    # Lemmatize
    stemming = PorterStemmer()
    txt = [stemming.stem(word) for word in txt]

    return " ".join(txt)

In [44]:
docs=df['Review'].apply(lambda x:preprocessing_text(x))

In [45]:
docs

0       manag becam agress said carbonara good rude ba...
1       order beef fillet ask done medium got well don...
2       attract venu welcom albeit somewhat slow servi...
3       sadli use high tripadvisor rate liter fortun e...
4       start meal bad especi given price visit husban...
                              ...                        
1497    despit review say love place hang especi beer ...
1498    beer good food awful decent thing shish kabob ...
1499    terribl servic truli comed level full pint bee...
1500    visit havana club museum locat old havana area...
1501       food servic aw pretti stop good photo bad food
Name: Review, Length: 1502, dtype: object

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

# Reduce max_features and adjust n-gram range
cv = CountVectorizer(max_features=6000, ngram_range=(1, 2))

# Create a vocabulary and word count vectors
word_count_vectors = cv.fit_transform(docs)

In [47]:
cv

CountVectorizer(max_features=6000, ngram_range=(1, 2))

In [48]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vectors)

TfidfTransformer()

In [56]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    # taking top items from vector
    sorted_items = sorted_items[:topn]
    
    score_vals = []
    feature_vals = []
    for idx, score in sorted_items:
        fname = feature_names[idx]
        score_vals.append(round(score, 3))
        feature_vals.append(fname)
    
    # create a tuple of feature, score
    results = {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]] = score_vals[idx]
    
    return results

# get feature names
feature_names = cv.get_feature_names_out()

def get_keywords(idx, docs):
    # generate tf-idf for the given document
    tf_idf_vector = tfidf_transformer.transform(cv.transform([docs[idx]]))

    # sort the tf-idf vectors by descending order of scores
    sorted_items = sort_coo(tf_idf_vector.tocoo())

    # extract only the top n; n here is 10
    keywords = extract_topn_from_vector(feature_names, sorted_items, 10)
    
    return keywords

def print_results(idx, keywords, df):
    # now print the results
    print("\n=====Sentiment=====")
    print(df['Sentiment'][idx])
    print("\n=====Review Title=====")
    print(df['Review Title'][idx])
    print("\n=====Review=====")
    print(df['Review'][idx])
    print("\n===Keywords===")
    for keyword, score in keywords.items():
        print(f"{keyword}: {score}")

# Example usage
idx = 1414
keywords = get_keywords(idx, docs)
print_results(idx, keywords, df)




=====Sentiment=====
Positive

=====Review Title=====
Music, music, outdoor cafes, balconies.....and history

=====Review=====
Loved this plaza...they had just fixed the fountain, and, of course, the famous gal on a rooster statue is here.  Latin music in all corners, some dancing, sometimes a stage with entertainment, always lively and SAFE at night.  Loved it!

===Keywords===
stage: 0.299
night love: 0.299
famou: 0.287
entertain: 0.287
statu: 0.271
danc: 0.271
fix: 0.264
safe: 0.259
sometim: 0.242
fountain: 0.242


In [57]:
import pickle
pickle.dump(tfidf_transformer,open('tfidf_transformer.pkl','wb'))
pickle.dump(cv,open('count_vectorizer.pkl','wb'))
pickle.dump(feature_names,open('feature_names.pkl','wb'))